Optimized RNA-Seq Processing with Error Handling and Configuration

This script processes RNA-Seq samples with optional truncation, alignment, quantification,
and coverage extraction. It supports error handling and configurable settings.

#Update sra-toolkit when needed

In [1]:
!apt-get remove --purge -y sra-toolkit

# Download the latest SRA Toolkit for Ubuntu 64-bit
!wget https://ftp-trace.ncbi.nlm.nih.gov/sra/sdk/3.1.1/sratoolkit.3.1.1-ubuntu64.tar.gz -O sratoolkit.tar.gz


# Extract the downloaded tar.gz file
!tar -xzf sratoolkit.tar.gz

import os

# Identify the extracted SRA Toolkit directory
sratoolkit_dir = [d for d in os.listdir() if d.startswith('sratoolkit') and d.endswith('ubuntu64')][0]

# Add the SRA Toolkit's bin directory to the PATH environment variable
os.environ['PATH'] += f":/content/{sratoolkit_dir}/bin"

# Verify that the PATH has been updated
print("Updated PATH:", os.environ['PATH'])

# Check the version of vdb-config
!vdb-config --version

# Check the version of fasterq-dump
!fasterq-dump --version

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Package 'sra-toolkit' is not installed, so not removed
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.
--2024-10-30 10:19:40--  https://ftp-trace.ncbi.nlm.nih.gov/sra/sdk/3.1.1/sratoolkit.3.1.1-ubuntu64.tar.gz
Resolving ftp-trace.ncbi.nlm.nih.gov (ftp-trace.ncbi.nlm.nih.gov)... 130.14.250.12, 130.14.250.13, 2607:f220:41e:250::10, ...
Connecting to ftp-trace.ncbi.nlm.nih.gov (ftp-trace.ncbi.nlm.nih.gov)|130.14.250.12|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93532905 (89M) [application/x-gzip]
Saving to: ‘sratoolkit.tar.gz’

sratoolkit.tar.gz   100%[===================>]  89.20M  82.6MB/s    in 1.1s    

2024-10-30 10:19:41 (82.6 MB/s) - ‘sratoolkit.tar.gz’ saved [93532905/93532905]

Updated PATH: /opt/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/content/sratoolkit.3.1.1-ubuntu64/b

#Import packages and connect to drive

In [2]:
import os
import subprocess
import multiprocessing
import shutil
import bisect
import psutil
import time
import json

In [3]:
# Number of logical CPUs (threads)
logical_cpus = psutil.cpu_count(logical=True)
print("Number of logical CPUs (threads):", logical_cpus)

# Number of physical CPU cores
physical_cpus = psutil.cpu_count(logical=False)
print("Number of physical CPU cores:", physical_cpus)

Number of logical CPUs (threads): 96
Number of physical CPU cores: 48


In [4]:
if not os.path.exists('/content/drive'):
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


# Install Necessary Tools

In [5]:
# Update and install required tools
subprocess.run(["apt-get", "update"])
subprocess.run(["apt-get", "install", "-y", "sra-toolkit", "samtools", "bedtools", "bedops", "libboost-all-dev"])

# Install STAR
subprocess.run(["wget", "https://github.com/alexdobin/STAR/archive/refs/tags/2.7.10a.zip"])
subprocess.run(["unzip", "2.7.10a.zip"])
subprocess.run(["make", "STAR"], cwd="STAR-2.7.10a/source")

# Install fastp
subprocess.run(["wget", "http://opengene.org/fastp/fastp", "-O", "/usr/local/bin/fastp"])
subprocess.run(["chmod", "+x", "/usr/local/bin/fastp"])

CompletedProcess(args=['chmod', '+x', '/usr/local/bin/fastp'], returncode=0)

In [6]:
# Create a directory to store the reference genome and annotation files
os.makedirs("/content/genome", exist_ok=True)

# Download reference genome
subprocess.run([
    "wget",
    "-O", "/content/genome/genome.fa.gz",
    "ftp://ftp.ensembl.org/pub/release-101/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz"
])
# Download annotation file
subprocess.run([
    "wget",
    "-O", "/content/genome/annotation.gtf.gz",
    "ftp://ftp.ensembl.org/pub/release-101/gtf/homo_sapiens/Homo_sapiens.GRCh38.101.gtf.gz"
])

# Unzip the files
subprocess.run(["gunzip", "/content/genome/genome.fa.gz"])
subprocess.run(["gunzip", "/content/genome/annotation.gtf.gz"])

# Define paths to genome and annotation files
genome_fasta = "/content/genome/genome.fa"
annotation_gtf = "/content/genome/annotation.gtf"

# Install Salmon and Build Index

In [7]:
# Install Salmon
subprocess.run(["wget", "https://github.com/COMBINE-lab/salmon/releases/download/v1.9.0/salmon-1.9.0_linux_x86_64.tar.gz"])
subprocess.run(["tar", "-xzf", "salmon-1.9.0_linux_x86_64.tar.gz"])
subprocess.run(["cp", "salmon-1.9.0_linux_x86_64/bin/salmon", "/usr/local/bin/"])

# Download cDNA sequences (transcriptome)
subprocess.run([
    "wget",
    "-O", "/content/genome/Homo_sapiens.GRCh38.cdna.all.fa.gz",
    "ftp://ftp.ensembl.org/pub/release-101/fasta/homo_sapiens/cdna/Homo_sapiens.GRCh38.cdna.all.fa.gz"
])

# Unzip the file
subprocess.run(["gunzip", "/content/genome/Homo_sapiens.GRCh38.cdna.all.fa.gz"])

# Build Salmon index
subprocess.run([
    "salmon",
    "index",
    "-t", "/content/genome/Homo_sapiens.GRCh38.cdna.all.fa",
    "-i", "/content/genome/salmon_index",
    "-k", "31"  # Default k-mer size is 31
])

CompletedProcess(args=['salmon', 'index', '-t', '/content/genome/Homo_sapiens.GRCh38.cdna.all.fa', '-i', '/content/genome/salmon_index', '-k', '31'], returncode=0)

#Picardtools setup

In [8]:
# Install Picard Tools
subprocess.run([
    "wget", "https://github.com/broadinstitute/picard/releases/download/3.2.0/picard.jar", "-O", "/content/picard.jar"
])

# Install Java
subprocess.run(["apt-get", "install", "-y", "openjdk-17-jdk-headless"])

os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-17-openjdk-amd64'
os.environ['PATH'] = os.environ['JAVA_HOME'] + '/bin:' + os.environ['PATH']

# Download gtfToGenePred from UCSC
subprocess.run("wget http://hgdownload.soe.ucsc.edu/admin/exe/linux.x86_64/gtfToGenePred -O /usr/local/bin/gtfToGenePred", shell=True)
subprocess.run("chmod +x /usr/local/bin/gtfToGenePred", shell=True)

# Generate refFlat file required by Picard
subprocess.run("gtfToGenePred /content/genome/annotation.gtf /content/genome/genes.genePred", shell=True)

# Corrected awk command for refFlat.txt
subprocess.run("awk 'BEGIN {OFS=\"\\t\"} {print $12, $1, $2, $3, $4, $5, $6, $7, $8, $9, $10, \".\", \".\"}' /content/genome/genes.genePred > /content/genome/refFlat.txt", shell=True)
subprocess.run("awk '{OFS=\"\\t\"; gene=($12 == \".\" || $12 == \"\") ? $1 : $12; print gene, $0}' /content/genome/genes.genePred > /content/genome/refFlat.txt", shell=True)

# Generate ribosomal intervals
subprocess.run("grep -i \"rRNA\" /content/genome/annotation.gtf > /content/genome/rRNA.gtf", shell=True)
subprocess.run("gtfToGenePred /content/genome/rRNA.gtf /content/genome/rRNA.genePred", shell=True)

# Convert GenePred to BED
subprocess.run("awk '{print $2\"\\t\"$4\"\\t\"$5}' /content/genome/rRNA.genePred > /content/genome/rRNA.bed", shell=True)

# Sort the BED file
subprocess.run("sort -k1,1 -k2,2n /content/genome/rRNA.bed > /content/genome/rRNA_sorted.bed", shell=True)

# Merge overlapping intervals
subprocess.run("bedtools merge -i /content/genome/rRNA_sorted.bed > /content/genome/rRNA_merged.bed", shell=True)

# Index the genome FASTA to get chromosome lengths
subprocess.run("samtools faidx /content/genome/genome.fa", shell=True)

# Start the IntervalList with the @HD header
subprocess.run("echo \"@HD\tVN:1.0\tSO:coordinate\" > /content/genome/ribosomal_intervals.list", shell=True)

# Add @SQ headers for each chromosome based on genome.fa.fai
subprocess.run("awk '{print \"@SQ\\tSN:\"$1\"\\tLN:\"$2}' /content/genome/genome.fa.fai >> /content/genome/ribosomal_intervals.list", shell=True)

# Convert merged BED to IntervalList entries and append to the list
subprocess.run("awk '{print $1\"\\t\"($2+1)\"\\t\"$3\"\\t+\\tRibosomalRNA\"}' /content/genome/rRNA_merged.bed >> /content/genome/ribosomal_intervals.list", shell=True)


CompletedProcess(args='awk \'{print $1"\\t"($2+1)"\\t"$3"\\t+\\tRibosomalRNA"}\' /content/genome/rRNA_merged.bed >> /content/genome/ribosomal_intervals.list', returncode=0)

#STAR Index creation

In [10]:
# Build index
subprocess.run([
    "./STAR-2.7.10a/source/STAR",
    "--runThreadN", "96",
    "--runMode", "genomeGenerate",
    "--genomeDir", "/content/star_genome_index",
    "--genomeFastaFiles", genome_fasta,
    "--sjdbGTFfile", annotation_gtf
])

CompletedProcess(args=['./STAR-2.7.10a/source/STAR', '--runThreadN', '96', '--runMode', 'genomeGenerate', '--genomeDir', '/content/star_genome_index', '--genomeFastaFiles', '/content/genome/genome.fa', '--sjdbGTFfile', '/content/genome/annotation.gtf'], returncode=0)

# Define Functions for Coverage Processing

In [11]:
def load_sorted_coverage(filepath):
    coverage_dict = {}
    with open(filepath, "r") as file:
        for line in file:
            chrom, pos, coverage = line.strip().split()
            pos = int(pos) - 1  # Convert to 0-based position
            if chrom not in coverage_dict:
                coverage_dict[chrom] = []
            coverage_dict[chrom].append((pos, int(coverage)))
    return coverage_dict

def filter_sequences_by_hard_min_coverage(input_file, output_file, hard_min_coverage):
    with open(input_file, "r") as infile, open(output_file, "w") as outfile:
        header, sequence, coverage_str = None, None, None
        for line in infile:
            line = line.strip()
            if line.startswith(">"):
                if header and sequence and coverage_str:
                    coverage_values = list(map(int, coverage_str.split(',')))
                    if all(c >= hard_min_coverage for c in coverage_values):
                        outfile.write(f"{header}\n{sequence}\n{coverage_str}\n")
                header, sequence, coverage_str = line, None, None
            elif not sequence:
                sequence = line
            else:
                coverage_str = line
        if header and sequence and coverage_str:
            coverage_values = list(map(int, coverage_str.split(',')))
            if all(c >= hard_min_coverage for c in coverage_values):
                outfile.write(f"{header}\n{sequence}\n{coverage_str}\n")
    print(f"Filtered sequences saved to {output_file}")

def adjust_fasta_headers(input_fasta, corrected_fasta):
    """
    Adjusts the start position in FASTA headers by incrementing it by 1.
    """
    try:
        with open(input_fasta, 'r') as infile, open(corrected_fasta, 'w') as outfile:
            for line in infile:
                line = line.rstrip('\n')
                if line.startswith('>'):
                    # Remove the initial '>' and split the header
                    header = line[1:].strip()
                    try:
                        chrom, positions = header.split(':')
                        start, end = positions.split('-')
                        new_start = int(start) + 1
                        # Construct the new header
                        new_header = f">{chrom}:{new_start}-{end}"
                        outfile.write(f"{new_header}\n")
                    except ValueError:
                        print(f"Warning: Malformed header skipped: {line.strip()}")
                        continue  # Skip malformed headers
                else:
                    # Write sequence and coverage lines unchanged
                    outfile.write(f"{line}\n")
        print(f"Adjusted headers saved to {corrected_fasta}")
    except FileNotFoundError:
        print(f"Error: The file {input_fasta} does not exist.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

def filter_fasta_by_chromosome(input_fasta, output_fasta, excluded_chromosomes):
    """
    Filters out sequences from the specified chromosomes in a FASTA file.

    Args:
        input_fasta (str): Path to the input FASTA file.
        output_fasta (str): Path to the output FASTA file with filtered sequences.
        excluded_chromosomes (list): List of chromosome identifiers to exclude.
    """
    with open(input_fasta, 'r') as infile, open(output_fasta, 'w') as outfile:
        write_sequence = False
        for line in infile:
            if line.startswith('>'):
                # Get the chromosome from the header
                header = line.strip()
                chrom_region = header[1:]  # Remove '>'
                if ':' in chrom_region:
                    chrom, _ = chrom_region.split(':', 1)
                else:
                    chrom = chrom_region
                if chrom in excluded_chromosomes:
                    write_sequence = False
                else:
                    write_sequence = True
                    outfile.write(line + '\n')
            else:
                if write_sequence:
                    outfile.write(line)

def fasta_to_bed(fasta_file, bed_file):
    """
    Convert the FASTA headers (regions) into a BED file.
    """
    with open(fasta_file, "r") as infile, open(bed_file, "w") as outfile:
        for line in infile:
            line = line.strip()
            if line.startswith(">"):
                # Parse the chromosome and region from the header (e.g., ">1:1000-2000")
                chrom_region = line[1:].strip()
                if ':' in chrom_region:
                    chrom, region = chrom_region.split(":")
                    start, end = region.split("-")
                    outfile.write(f"{chrom}\t{int(start) - 1}\t{end}\n")  # Convert to 0-based start for BED
    print(f"Regions have been converted to BED format and saved to {bed_file}")


# Define the Per-Sample Processing Function

In [16]:
def process_sample(sample, adjusted_read_length, library_layout, filter_sequences=False, truncate=False, truncation_range=(50, 100), truncation_step=10,
                   run_star=True, run_picard=True, run_salmon=True, threads_per_sample=16, regions_fasta_file='path',
                   drive_metrics_dir='/content/drive/My Drive/Colab/Bryan_Colab'):
    """
    Processes a single RNA-Seq sample by performing the following steps:
    If truncate is True:
        For each truncation length:
            1. Truncate reads to different lengths using fastp.
            2. Aligns truncated reads with STAR.
            3. Indexes the BAM file with samtools.
            4. Collects RNA-Seq metrics using Picard.
            5. Runs Salmon quantification.
            6. Uses regions from another dataset, excluding chromosome X sequences.
            7. Extracts coverage values and processes sequences.
            8. Copies the metrics and processed data to Google Drive.
            9. Deletes intermediate files to free up storage.
    If truncate is False:
        1. Use fastp for adapter removal only.
        2. Proceed with the rest of the processing steps without truncation.
        3. Extract read length information from the fastp report.

    Args:
        sample (str): The sample name (e.g., "SRR10072432").
        adjusted_read_length (float): Adjusted read length after considering paired-end reads.
        library_layout (str): 'PAIRED' or 'SINGLE' indicating the library layout.
        filter_sequences (bool): Flag to indicate whether to filter sequences.
        truncate (bool): Flag to control whether to perform truncation.
        truncation_range (tuple): Tuple specifying the start and end of truncation lengths.
        truncation_step (int): Step size for truncation lengths.
        run_star (bool): Flag to control whether to run STAR alignment.
        run_picard (bool): Flag to control whether to run Picard metrics collection.
        run_salmon (bool): Flag to control whether to run Salmon quantification.
        threads_per_sample (int): Number of threads to use per sample.
        regions_fasta_file (str): Path to the regions FASTA file.
        drive_metrics_dir (str): Directory on Google Drive to save outputs.
    """
    print(f"Processing sample: {sample}\n")
    start_time = time.time()

    # Set Java environment variables within the child process
    os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-17-openjdk-amd64'
    os.environ['PATH'] = os.environ['JAVA_HOME'] + '/bin:' + os.environ['PATH']

    # Create a sample-specific directory
    sample_dir = f"/content/{sample}"
    if not os.path.exists(sample_dir):
        os.makedirs(sample_dir)

    # Download Sample Data from SRA using fastq-dump
    try:
        subprocess.run([
            "fastq-dump",
            "--split-files",
            "--gzip",
            "-O", sample_dir,
            sample  # Use the sample name directly
        ], check=True)
    except subprocess.CalledProcessError as e:
        print(f"fastq-dump failed for {sample}: {e}")
        return  # Skip processing this sample

    # Determine input FASTQ file based on library layout
    if library_layout == 'PAIRED':
        input_fastq = f"{sample_dir}/{sample}_1.fastq.gz"
    else:
        input_fastq = f"{sample_dir}/{sample}.fastq.gz"

    # Check if input_fastq exists
    if not os.path.exists(input_fastq):
        print(f"Input FASTQ file not found for {sample}")
        return  # Skip processing this sample

    # Verify downloaded files
    print(f"Downloaded files for {sample}:")
    subprocess.run(["ls", sample_dir])

    # Define paths to genome and annotation files
    genome_fasta = "/content/genome/genome.fa"
    annotation_gtf = "/content/genome/annotation.gtf"

    # Define paths to Picard input and output
    ref_flat = "/content/genome/refFlat.txt"
    ribosomal_intervals = "/content/genome/ribosomal_intervals.list"

    # Determine truncation lengths
    if truncate:
        truncation_lengths = list(range(truncation_range[0], truncation_range[1]+1, truncation_step))
    else:
        truncation_lengths = [None]  # No truncation

    for length in truncation_lengths:
        try:
            if length is not None:
                # Truncation Mode: Run fastp with truncation
                length_str = str(length)
                print(f"\nProcessing truncation length: {length_str}")

                # Define the output FASTQ file for truncated reads
                fastq_output = f"{sample_dir}/{sample}_truncated_{length_str}.fastq.gz"

                # Define the output JSON report file
                json_report = f"{sample_dir}/fastp_report_{length_str}.json"

                # Run fastp with truncation and adapter removal
                fastp_cmd = [
                    "fastp",
                    "-i", input_fastq,
                    "-o", fastq_output,
                    "--cut_right",
                    "--max_len1", length_str,
                    "--length_required", length_str,
                    "-j", json_report,
                ]
            else:
                # Non-Truncation Mode: Run fastp for adapter removal only
                length_str = str(int(adjusted_read_length))
                print(f"\nProcessing original sample without truncation, adjusted read length: {length_str}")

                # Define the output FASTQ file for adapter-removed reads
                fastq_output = f"{sample_dir}/{sample}_fastp_processed.fastq.gz"

                # Define the output JSON report file
                json_report = f"{sample_dir}/fastp_report_{length_str}.json"

                # Run fastp without truncation, only adapter removal
                fastp_cmd = [
                    "fastp",
                    "-i", input_fastq,
                    "-o", fastq_output,
                    "-j", json_report,
                ]

            # Execute fastp and capture its output
            try:
                result = subprocess.run(
                    fastp_cmd,
                    check=True,
                    stdout=subprocess.PIPE,
                    stderr=subprocess.PIPE,
                    text=True
                )
                print(f"fastp output for {sample} at length {length_str}:\n{result.stdout}")
                if result.stderr:
                    print(f"fastp stderr for {sample} at length {length_str}:\n{result.stderr}")
            except subprocess.CalledProcessError as e:
                print(f"fastp failed for {sample} at length {length_str}: {e}")
                print(f"fastp stdout: {e.stdout}")
                print(f"fastp stderr: {e.stderr}")
                continue  # Skip to next length

            # Set fastq_input to the output of fastp
            fastq_input = fastq_output

            # After running fastp and before proceeding to alignment
            try:
                # Read the fastp JSON report
                with open(json_report, 'r') as f:
                    fastp_report = json.load(f)

                # Extract the read length from total_cycles
                read_length = fastp_report['read1_after_filtering']['total_cycles']

                # Extract total_bases after filtering
                total_bases = fastp_report['summary']['after_filtering']['total_bases']

                # Print or save the depth information
                print(f"Total bases (depth) after filtering: {total_bases}")

                # Calculate depth in Mbases for readability
                depth_mb = round(total_bases / 1e6, 2)
                print(f"Depth in Mbases: {depth_mb} Mb")

                # Save read length and depth information to a file
                read_length_info_file = f"{sample_dir}/{sample}_{length_str}_read_length_info.txt"
                with open(read_length_info_file, 'w') as f:
                    f.write(f"Read length after filtering: {read_length}\n")
                    f.write(f"Total bases (depth) after filtering: {total_bases}\n")
                    f.write(f"Depth in Mbases: {depth_mb} Mb\n")

                # Copy the read length info to Google Drive
                #drive_read_length_info_file = os.path.join(drive_metrics_dir, f'{sample}_{length_str}_read_length_info.txt')
                #shutil.copy(read_length_info_file, drive_read_length_info_file)
                #print(f"Copied read length info to Google Drive: {drive_read_length_info_file}")
            except Exception as e:
                print(f"Failed to extract read length information for {sample} at length {length_str}: {e}")

            bam_file = None

            # Align reads with STAR
            if run_star:
                try:
                    # Align reads with STAR without passing Read Group lines
                    cpu_count = psutil.cpu_count(logical=True)
                    threads = min(threads_per_sample, cpu_count)
                    subprocess.run([
                        "./STAR-2.7.10a/source/STAR",
                        "--runThreadN", str(threads),
                        "--genomeDir", "/content/star_genome_index",
                        "--readFilesIn", fastq_input,
                        "--outFileNamePrefix", f"{sample_dir}/{sample}_output_STAR_{length_str}",
                        "--readFilesCommand", "zcat",
                        "--outSAMtype", "BAM", "SortedByCoordinate"
                    ], check=True)
                    # Path to the aligned BAM file
                    bam_file = f"{sample_dir}/{sample}_output_STAR_{length_str}Aligned.sortedByCoord.out.bam"
                    # Index the BAM file using samtools
                    subprocess.run([
                        "samtools", "index", bam_file
                    ], check=True)
                except subprocess.CalledProcessError as e:
                    print(f"STAR alignment failed for {sample} at length {length_str}: {e}")
                    continue  # Skip to next length
            else:
                print(f"Skipping STAR alignment for {sample} at length {length_str}")

            # Run Picard CollectRnaSeqMetrics
            if run_picard and bam_file:
                try:
                    metrics_output = f"{sample_dir}/{sample}_{length_str}_Picard_RNA_Metrics.txt"
                    chart_output = f"{sample_dir}/{sample}_{length_str}_Picard_RNA_Metrics.pdf"
                    picard_cmd = [
                        "java", "-jar", "/content/picard.jar", "CollectRnaSeqMetrics",
                        f"I={bam_file}",
                        f"O={metrics_output}",
                        f"REF_FLAT={ref_flat}",
                        f"RIBOSOMAL_INTERVALS={ribosomal_intervals}",
                        "STRAND=NONE",
                        f"CHART_OUTPUT={chart_output}",
                        f"R={genome_fasta}"
                    ]
                    result = subprocess.run(picard_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
                    if result.returncode != 0:
                        print(f"Picard CollectRnaSeqMetrics failed for {sample} at length {length_str}")
                        print(f"Picard stdout: {result.stdout}")
                        print(f"Picard stderr: {result.stderr}")
                        continue
                    else:
                        print(f"Picard CollectRnaSeqMetrics completed successfully for {sample} at length {length_str}")
                    # Copy the metrics files to Google Drive
                    drive_metrics_file = os.path.join(drive_metrics_dir, f'{sample}_{length_str}_Picard_RNA_Metrics.txt')
                    drive_metrics_file_pdf = os.path.join(drive_metrics_dir, f'{sample}_{length_str}_Picard_RNA_Metrics.pdf')
                    shutil.copy(metrics_output, drive_metrics_file)
                    shutil.copy(chart_output, drive_metrics_file_pdf)
                    print(f"Copied RNA-Seq metrics to Google Drive: {drive_metrics_file}")
                except Exception as e:
                    print(f"Picard CollectRnaSeqMetrics failed for {sample} at length {length_str}: {e}")
                    continue  # Skip to next length
            else:
                print(f"Skipping Picard CollectRnaSeqMetrics for {sample} at length {length_str}")

            # Run Salmon Quantification
            if run_salmon:
                try:
                    salmon_output_dir = f"{sample_dir}/salmon_quant_{length_str}"
                    subprocess.run([
                        "salmon",
                        "quant",
                        "-i", "/content/genome/salmon_index",
                        "-l", "A",  # Automatically detect library type
                        "-r", fastq_input,
                        "-p", str(threads_per_sample),
                        "-o", salmon_output_dir
                    ], check=True)
                    print(f"Salmon quantification completed for {sample} at length {length_str}")
                    # Copy Salmon outputs to Google Drive
                    drive_salmon_output_dir = os.path.join(drive_metrics_dir, f'{sample}_salmon_quant_{length_str}')
                    shutil.copytree(salmon_output_dir, drive_salmon_output_dir)
                    print(f"Copied Salmon quantification results to Google Drive: {drive_salmon_output_dir}")
                except subprocess.CalledProcessError as e:
                    print(f"Salmon quantification failed for {sample} at length {length_str}: {e}")
                    continue  # Skip to next length
            else:
                print(f"Skipping Salmon quantification for {sample} at length {length_str}")

            # If bam_file exists, proceed with coverage extraction
            if bam_file:
                try:
                    """# Use Regions from Another Dataset, Excluding Chromosome X"""
                    # Use regions_fasta_file parameter
                    # Exclude sequences from chromosome X
                    excluded_chromosomes = ['X']
                    filtered_regions_fasta_file = f"{sample_dir}/regions_from_other_dataset_filtered.fa"
                    filter_fasta_by_chromosome(regions_fasta_file, filtered_regions_fasta_file, excluded_chromosomes)

                    # Convert the regions from the filtered dataset to BED
                    bed_file = f"{sample_dir}/regions_from_other_dataset_{length_str}.bed"
                    fasta_to_bed(filtered_regions_fasta_file, bed_file)

                    # Extract sequences from genome for these regions
                    sequences_file = f"{sample_dir}/extracted_sequences_{length_str}.fa"
                    subprocess.run([
                        "bedtools", "getfasta",
                        "-fi", genome_fasta,
                        "-bed", bed_file,
                        "-fo", sequences_file
                    ], check=True)

                    """# Coverage Extraction and Sequence Processing"""
                    # Define output coverage file
                    coverage_output = f"{sample_dir}/coverage_per_base_{length_str}.samtools.bed"
                    # Calculate coverage using the BAM file for the current length
                    with open(coverage_output, "w") as cov_out:
                        subprocess.run(["samtools", "depth", "-b", bed_file, bam_file], stdout=cov_out, check=True)

                    # Sort the coverage data
                    sorted_coverage_output = f"{sample_dir}/sorted_coverage_per_base_{length_str}.samtools.bed"
                    with open(sorted_coverage_output, "w") as sorted_cov_out:
                        subprocess.run(["sort", "-k1,1", "-k2,2n", coverage_output], stdout=sorted_cov_out, check=True)

                    # Load the coverage data
                    coverage_data = load_sorted_coverage(sorted_coverage_output)

                    # Define output file for sequences with coverage
                    output_file = f"{sample_dir}/{sample}_{length_str}_enhanced_sequences_with_coverage.fa"

                    # Open the sequences file and process
                    with open(sequences_file, "r") as fasta_file, open(output_file, "w") as outfile:
                        header = None
                        sequence = ""

                        for line in fasta_file:
                            line = line.strip()
                            if line.startswith(">"):
                                if header and sequence:
                                    chrom_region = header[1:]
                                    if ':' in chrom_region:
                                        chrom, region = chrom_region.split(":")
                                        start, end = map(int, region.split("-"))
                                    else:
                                        chrom = chrom_region
                                        start, end = 0, len(sequence)
                                    coverage_list = coverage_data.get(chrom, [])
                                    start_positions = [pos for pos, cov in coverage_list]
                                    idx = bisect.bisect_left(start_positions, start)

                                    coverage_values = []
                                    for i in range(len(sequence)):
                                        current_position = start + i
                                        if idx < len(coverage_list) and coverage_list[idx][0] == current_position:
                                            coverage_values.append(coverage_list[idx][1])
                                            idx += 1
                                        else:
                                            coverage_values.append(0)
                                    coverage_str = ','.join(map(str, coverage_values))
                                    outfile.write(f"{header}\n{sequence}\n{coverage_str}\n")
                                header = line
                                sequence = ""
                            else:
                                sequence += line

                        # Process the last sequence
                        if header and sequence:
                            chrom_region = header[1:]
                            if ':' in chrom_region:
                                chrom, region = chrom_region.split(":")
                                start, end = map(int, region.split("-"))
                            else:
                                chrom = chrom_region
                                start, end = 0, len(sequence)
                            coverage_list = coverage_data.get(chrom, [])
                            start_positions = [pos for pos, cov in coverage_list]
                            idx = bisect.bisect_left(start_positions, start)

                            coverage_values = []
                            for i in range(len(sequence)):
                                current_position = start + i
                                if idx < len(coverage_list) and coverage_list[idx][0] == current_position:
                                    coverage_values.append(coverage_list[idx][1])
                                    idx += 1
                                else:
                                    coverage_values.append(0)
                            coverage_str = ','.join(map(str, coverage_values))
                            outfile.write(f"{header}\n{sequence}\n{coverage_str}\n")
                    print(f"Sequences with coverage data saved to {output_file}")

                    if filter_sequences:
                        # Filter sequences based on hard minimum coverage
                        filtered_output_file = f"{sample_dir}/{sample}_{length_str}_filtered_high_quality_sequences.fa"
                        filter_sequences_by_hard_min_coverage(output_file, filtered_output_file, hard_min_coverage=10)

                        # Adjust the headers and save to a new file
                        corrected_fasta = filtered_output_file.replace('.fa', '_corrected.fa')

                        # Adjust the headers
                        adjust_fasta_headers(filtered_output_file, corrected_fasta)

                        # Define the destination path on Google Drive with depth in filename
                        drive_corrected_file = os.path.join(
                            drive_metrics_dir,
                            f'{sample}_readlength_{length_str}_depth_{depth_mb}Mb.fa'
                        )
                    else:
                        # No filtering; use the original output_file
                        corrected_fasta = output_file.replace('.fa', '_corrected.fa')

                        # Adjust the headers
                        adjust_fasta_headers(output_file, corrected_fasta)

                        # Define the destination path on Google Drive with depth in filename
                        drive_corrected_file = os.path.join(
                            drive_metrics_dir,
                            f'{sample}_readlength_{length_str}_depth_{depth_mb}Mb.fa'
                        )

                    # Copy the corrected FASTA file to Google Drive
                    shutil.copy(corrected_fasta, drive_corrected_file)
                    print(f"Copied corrected FASTA to Google Drive: {drive_corrected_file}")
                except Exception as e:
                    print(f"Error during coverage extraction and sequence processing for {sample} at length {length_str}: {e}")
                    continue  # Skip to next length
            else:
                print(f"No BAM file available, skipping coverage extraction and sequence processing for {sample} at length {length_str}")

            # Delete intermediate files to free up storage
            try:
                if length is not None:
                    os.remove(fastq_input)  # Remove truncated or processed FASTQ
                if bam_file:
                    os.remove(bam_file)
                    os.remove(f"{bam_file}.bai")
                if run_picard and bam_file:
                    os.remove(metrics_output)
                    os.remove(chart_output)
                if 'coverage_output' in locals():
                    os.remove(coverage_output)
                if 'sorted_coverage_output' in locals():
                    os.remove(sorted_coverage_output)
                if 'output_file' in locals():
                    os.remove(output_file)
                if 'corrected_fasta' in locals():
                    os.remove(corrected_fasta)
                if 'sequences_file' in locals():
                    os.remove(sequences_file)
                if 'bed_file' in locals():
                    os.remove(bed_file)
                if 'filtered_regions_fasta_file' in locals():
                    os.remove(filtered_regions_fasta_file)
                if 'read_length_info_file' in locals():
                    os.remove(read_length_info_file)
                # Do not delete Salmon outputs if you need them
                # Uncomment the next line if you want to delete Salmon outputs
                # if run_salmon and 'salmon_output_dir' in locals():
                #     shutil.rmtree(salmon_output_dir)
                print(f"Deleted intermediate files for length {length_str}")
            except Exception as e:
                print(f"Error deleting intermediate files for length {length_str}: {e}")

        except Exception as e:
            print(f"Processing failed for {sample} at length {length_str}: {e}")
            continue  # Skip to next length

    # Delete the original FASTQ files
    try:
        if library_layout == 'PAIRED':
            os.remove(f"{sample_dir}/{sample}_1.fastq.gz")
            os.remove(f"{sample_dir}/{sample}_2.fastq.gz")
        else:
            os.remove(f"{sample_dir}/{sample}.fastq.gz")
        print(f"Deleted original FASTQ files for {sample}")
    except Exception as e:
        print(f"Error deleting original FASTQ files for {sample}: {e}")

    # Delete the sample directory to free up storage
    try:
        shutil.rmtree(sample_dir)
        print(f"Deleted sample directory {sample_dir} to free up storage.")
    except Exception as e:
        print(f"Error deleting sample directory {sample_dir}: {e}")

    end_time = time.time()
    print(f"\nAll processing steps completed for {sample}.")
    print(f"Data preprocessing took {end_time - start_time:.2f} seconds\n")

# Parallel Processing of Multiple Samples

In [ ]:
if __name__ == '__main__':
    import multiprocessing
    import time
    from math import ceil
    import pandas as pd
    import numpy as np

    # ===== Parameter Configurations =====
    # Flags
    filter_sequences = False       # Set to False to skip filtering
    truncate = False                # Set to True or False
    run_star = True
    run_picard = True
    run_salmon = False

    # Truncation parameters
    truncation_range = (50, 150)    # Set the truncation range if truncate is True
    truncation_step = 25            # Set the truncation step if truncate is True

    # Threading
    threads_per_sample = 16         # Adjust based on your environment

    # File paths
    regions_fasta_file = "/content/drive/My Drive/Colab/Bryan_Colab/SRR10072432_1_filtered_high_quality_sequences_150bp.fa"
    drive_metrics_dir = '/content/drive/My Drive/Colab/Bryan_Colab'

    # Sampling parameters
    csv_path = '/content/drive/My Drive/Colab/Bryan_Colab/srr_metadata_scrape_filtered.csv'
    num_samples = 160                 # You can adjust this number as needed

    # Binning parameters
    num_bins = 21                    # Number of bins between 50 and 150 (if you want 2 bins -> value needs to be 3)
    bin_start, bin_end = 50, 150
    bins = np.linspace(bin_start, bin_end, num=num_bins+1)  # e.g., 11 edges for 10 bins

    # ===== Load and Process CSV =====
    df = pd.read_csv(csv_path)

    # Adjust ReadLength for paired-end reads
    def adjust_read_length(row):
        if row['LibraryLayout'].upper() == 'PAIRED':
            return row['ReadLength'] / 2
        else:
            return row['ReadLength']

    df['AdjustedReadLength'] = df.apply(adjust_read_length, axis=1)

    # Filter samples to AdjustedReadLength between 50 and 150
    df_filtered = df[(df['AdjustedReadLength'] >= bin_start) & (df['AdjustedReadLength'] <= bin_end)]

    # Assign bins
    df_filtered['ReadLengthBin'] = pd.cut(df_filtered['AdjustedReadLength'], bins=bins, include_lowest=True)

    # Now, group by 'ReadLengthBin' and randomly sample from each bin
    grouped = df_filtered.groupby('ReadLengthBin')

    samples_per_bin = max(1, num_samples // len(grouped))  # Ensure at least one sample per bin

    sampled_dfs = []

    for name, group in grouped:
        n = min(samples_per_bin, len(group))  # Don't sample more than available
        sampled_dfs.append(group.sample(n=n, random_state=42))

    sampled_df = pd.concat(sampled_dfs)

    # If we have more samples than needed, randomly select the required number
    if len(sampled_df) > num_samples:
        sampled_df = sampled_df.sample(n=num_samples, random_state=42)

    # Prepare a list of arguments for process_sample
    process_args = []

    for idx, row in sampled_df.iterrows():
        sample = row['Run']
        adjusted_read_length = row['AdjustedReadLength']
        library_layout = row['LibraryLayout']
        process_args.append((
            sample,
            adjusted_read_length,
            library_layout,
            filter_sequences,
            truncate,
            truncation_range,
            truncation_step,
            run_star,
            run_picard,
            run_salmon,
            threads_per_sample,
            regions_fasta_file,
            drive_metrics_dir
        ))

    # Number of parallel processes (adjust based on available cores and desired threads per sample)
    cpu_available = multiprocessing.cpu_count()
    num_processes = min(len(process_args), cpu_available // threads_per_sample)
    num_processes = max(1, num_processes)  # Ensure at least one process

    # Function to batch samples
    def batch_samples(args_list, batch_size):
        """Yield successive batches from args_list."""
        for i in range(0, len(args_list), batch_size):
            yield args_list[i:i + batch_size]

    # Process samples in batches
    total_batches = ceil(len(process_args) / num_processes)
    batch_number = 1

    for arg_batch in batch_samples(process_args, num_processes):
        print(f"\nProcessing batch {batch_number} of {total_batches} with {len(arg_batch)} samples...")
        start_batch_time = time.time()

        # Create a pool of worker processes
        pool = multiprocessing.Pool(processes=len(arg_batch))

        # Map the process_sample function to the batch of samples with the new parameters
        pool.starmap(process_sample, arg_batch)

        # Close the pool and wait for the work to finish
        pool.close()
        pool.join()

        end_batch_time = time.time()
        print(f"Batch {batch_number} completed in {end_batch_time - start_batch_time:.2f} seconds.\n")
        batch_number += 1


Processing batch 1 of 24 with 6 samples...


<ipython-input-17-aa2ab59ffc05>:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['ReadLengthBin'] = pd.cut(df_filtered['AdjustedReadLength'], bins=bins, include_lowest=True)
<ipython-input-17-aa2ab59ffc05>:55: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df_filtered.groupby('ReadLengthBin')


Processing sample: SRR6195892
Processing sample: SRR5356297
Processing sample: SRR1531494
Processing sample: SRR1955042
Processing sample: SRR21391238
Processing sample: SRR960412






Input FASTQ file not found for SRR1955042
Downloaded files for SRR1531494:

Processing original sample without truncation, adjusted read length: 50
Downloaded files for SRR6195892:

Processing original sample without truncation, adjusted read length: 50
fastp output for SRR1531494 at length 50:

fastp stderr for SRR1531494 at length 50:
Detecting adapter sequence for read1...
No adapter detected for read1

Read1 before filtering:
total reads: 35757452
total bases: 1787872600
Q20 bases: 1759906961(98.4358%)
Q30 bases: 1708567549(95.5643%)

Read1 after filtering:
total reads: 35586822
total bases: 1779341100
Q20 bases: 1755963157(98.6861%)
Q30 bases: 1705248685(95.836%)

Filtering result:
reads passed filter: 35586822
reads failed due to low quality: 170630
reads failed due to too many N: 0
reads failed d

In [ ]:
from IPython.display import display, Javascript

# This command disconnects the current runtime
display(Javascript('google.colab.kernel.disconnect()'))
